### Informações sobre atributos:
1) __id__: identificador único
2) __gênero__: "Masculino", "Feminino" ou "Outro"
3) __idade__: idade do paciente
4) __hipertensão__: 0 se o paciente não tiver hipertensão, 1 se o paciente tiver hipertensão
5) __doença cardíaca__: 0 se o paciente não tiver doença cardíaca, 1 se o paciente tiver doença cardíaca
6) __já casado__: "Não" ou "Sim"
7) __tipo de trabalho__: "crianças"(`children`), "Funcionário público"(`Govt_jov`), "Nunca trabalhou"(`Never_worked`), "Privado"(`Private`) ou "Autônomo"(`Self-employed`)
8) __tipo de residência__: "Rural" ou "Urbana"
9) __nível médio de glicose__: nível médio de glicose no sangue
10) __IMC__: índice de massa corporal
11) __status de tabagismo__: "ex-fumante"(`formerly smoked`), "nunca fumou"(`formerly smoked`), "fuma"(`smokes`) ou "Desconhecido"(`Unknown`)
12) __AVC__: 1 se o paciente teve um AVC ou 0 se não teve

In [ ]:
import copy 
import scipy
import warnings
import pandas as pd
import numpy as np
import seaborn as sea 
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
warnings.filterwarnings('ignore',category=FutureWarning)

In [ ]:
data = pd.read_csv(r'datasets/StrokePredictionDataset/healthcare-dataset-stroke-data.csv',sep=',')

In [ ]:
data.head(4)

## Analise dos dados

In [ ]:
data.describe()

> Nesse conjunto de daddos para valores númericos continuos irei fazer a padronização dos dados

In [ ]:
data.nunique()

> quantidade de valores unicos de cada atributo

In [ ]:
valoresNulos = (pd.concat([data.isna().nunique(),data.isnull().nunique()] ,axis=1))
valoresNulos.columns = ['NaN','NULL']
valoresNulos

In [ ]:
data.loc[data['bmi'].isna()==True]['stroke'].value_counts()

In [ ]:
data.drop(index = data.loc[data['bmi'].isna()==True].index , inplace=True)

> Será removido para a analises depois decido oq fazer com esses valores nulos

In [ ]:
pizza = data['stroke'].value_counts()

plt.figure(figsize=(6,4),facecolor='#FAFAD2')
plt.pie(x=pizza.values , labels= ['Não teve AVC','Teve AVC'] , autopct = lambda x:'{:.2f}({:.0f})'.format(x,((x*pizza.sum())/100)) , colors=sea.color_palette('coolwarm',n_colors=2),shadow=True,explode=(0,0.045),pctdistance=0.6)
plt.title('Proporção dos dados')
plt.show()

> No preprocessamento dos dados para a classificação se faz necessario o balanceamento dos dados , a mais valores para pessoas que não teve Avc para as que tiveram

In [ ]:
sea.boxplot( data[['bmi','age','avg_glucose_level','stroke']])
plt.grid(True)
plt.show()

In [ ]:
fig , ax = plt.subplots(1,3)
fig.set_figwidth(15)
colors   = sea.color_palette('dark:salmon_r',n_colors=3)
index    = 0
for coluna in ['bmi','age','avg_glucose_level']:
    sea.histplot(data[coluna], ax=ax[index] ,color=colors[index] , kde=True); index +=1
plt.show()

> Há outlier

In [ ]:
labels = ['Baixo peso','Peso ideal','Sobrepeso','Obesidade 1','Obesidade 2(sévera)','Obesidade 3(mórbida)']
bins   = [min(data['bmi']),18.5,24.9,29.9,34.9,39.9,max(data['bmi']) ]
freq  = pd.cut(x = data['bmi'] , bins=bins ,labels=labels)
hist  = pd.concat([freq,data[['stroke']]],axis=1)
hist  = hist.groupby(by=['bmi'])['stroke'].value_counts().reset_index()

In [ ]:
fig ,ax = plt.subplots(1,1)
fig.set_figwidth(12)
fig.set_figheight(6)

colors  = ['#6495ED','#DC143C'] 
bar = sea.barplot(x = hist['bmi'] ,y=hist['count'],hue=hist['stroke'],palette=colors ,ax=ax)
bar.bar_label( bar.containers[0] )
bar.bar_label( bar.containers[1] )

ax.set_ylabel('Quantidade')
ax.set_xlabel('Tipo de peso corporal')
ax.spines[['top','left']].set_visible(False)
legends = []
for value, color in zip(['Não teve','Teve'], colors): legends.append(mpatches.Patch(color=color, label=value))

ax.legend(title='Avc',handles=legends,bbox_to_anchor=(1,0.8,0,0))
plt.grid(True)
plt.show()

In [ ]:
pizza = freq.value_counts()

plt.figure(figsize=(8,5),facecolor='#FAFAD2')
plt.pie(x=pizza.values , labels= pizza.index , autopct = lambda x:'{:.2f}({:.0f})'.format(x,((x*pizza.sum())/100)) , colors=sea.color_palette('coolwarm',n_colors=6),shadow=True,pctdistance=0.69,explode=(0,0,0,0.03,0.05,0.06))
plt.title('Proporção dos dados')
plt.show()

In [ ]:
data['gender'].value_counts().reset_index()

In [ ]:
data.drop( index = data.loc[data['gender']=='Other'].index , inplace=True)

> No atributo `gender` o valores other só aparece uma vez , será removido

In [ ]:
pizza = data['gender'].value_counts()

plt.figure(figsize=(6,4))
plt.pie(x = pizza.values , labels = pizza.index , explode=(0,0.04),autopct=lambda x: '{:.2f}({:.0f})'.format(x,((x*sum(pizza.values))/100)) , shadow=True)
plt.title('Proporção de valores do atributo gender')
plt.show()

In [ ]:
resultado = data.groupby(by=['gender','stroke'])['age'].mean().reset_index()

fig , ax = plt.subplots(1,1)
fig.set_figwidth(8)
fig.set_figheight(6)

bar = sea.barplot(x = resultado['stroke'] , y=resultado['age'] ,hue = resultado['gender'],palette=sea.color_palette('pastel')[0:2][::-1],width=0.5,ax=ax)
bar.bar_label(bar.containers[0], fontsize=10)
bar.bar_label(bar.containers[1], fontsize=10)

ax.set_axisbelow(True)
ax.grid(axis='y')
ax.spines[['right','top','left']].set_visible(False)
plt.legend(title='Gênero')
plt.title('Média de idade dos gêneros que teve ou não avc',y=1.029)
plt.xticks([0,1],['Não teve','Teve'])
plt.ylabel('Média de idade')
plt.xlabel('AVC')

plt.show()

In [ ]:
labels = ['Baixo peso','Peso ideal','Sobrepeso','Obesidade 1','Obesidade 2(sévera)','Obesidade 3(mórbida)']
bins   = [min(data['bmi']),18.5,24.9,29.9,34.9,39.9,max(data['bmi']) ]

freq  = pd.cut(x = data['bmi'] , bins=bins ,labels=labels)

resultado = pd.concat([freq,data[['work_type','stroke']]],axis=1)

In [ ]:
gropbyresultado = resultado.groupby(by=['work_type'])['bmi'].value_counts().reset_index()

In [ ]:
gropbyresultado

In [ ]:

sea.set_theme(style="whitegrid")
fig , ax = plt.subplots(1,1)
fig.set_figwidth(12)
fig.set_figheight(8)

palette = sea.color_palette("muted",n_colors=4)
bar = sea.barplot(x=gropbyresultado['count'],y=gropbyresultado['work_type'],hue=gropbyresultado['bmi'],palette=palette,ax=ax)
for p in bar.patches:
    if p.get_width() != 0:
        bar.annotate("%.0f" % p.get_width(),  xy = (p.get_width(), p.get_y()+p.get_height()/2), 
        xytext = (0, 0.3), textcoords= 'offset points', ha = 'left', va = "center" ,size=10)
ax.spines[['right','top','bottom']].set_visible(False)
ax.set_ylabel('Tipo de trabalho')
ax.set_xticks([])
ax.set_xlabel('')
ax.set_axisbelow(True)
ax.grid(True)
ax.set_title('Tipo de trabalho e a quantidade pessoas que tem um determinado tipo de peso',y=1.029)
ax.legend().set_title('Tipo de peso')
plt.show()

In [ ]:
resultado = pd.concat([data['gender'],freq],axis=1)
resultado = resultado.groupby(by='bmi')['gender'].value_counts().reset_index()

In [ ]:
plt.figure(figsize=(13,6))
sea.set_theme(style='whitegrid')
bar = sea.barplot(x = resultado['bmi'],y=resultado['count'],hue=resultado['gender'],palette=sea.color_palette('pastel',n_colors=2)[::-1])
bar.bar_label(bar.containers[0])
bar.bar_label(bar.containers[1])
plt.ylabel('Quantidade')
plt.xlabel('Tipo de peso corporal')
plt.show()

In [ ]:
densidade = data[['avg_glucose_level','bmi','stroke']]

fig , ax = plt.subplots(1,2)
fig.set_figwidth(10)
fig.set_figheight(5)

fig.subplots_adjust(wspace=0.5)
sea.violinplot(x = densidade['stroke'],y = densidade['avg_glucose_level'] ,ax=ax[0])
sea.violinplot(x = densidade['stroke'],y = densidade['bmi'],ax=ax[1])
plt.title('Distribuição dos dados')
plt.show()

In [ ]:
sea.pairplot(data,hue='stroke')
plt.show()

&nbsp;

&nbsp;

# Pré-processamento

In [ ]:
from sklearn.preprocessing   import LabelBinarizer,StandardScaler,MinMaxScaler,LabelEncoder
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling  import RandomOverSampler
from sklearn.metrics.pairwise import paired_distances
from sklearn.impute import SimpleImputer

In [ ]:
dataset = pd.read_csv(r'datasets/StrokePredictionDataset/healthcare-dataset-stroke-data.csv')

In [ ]:
dataset.head(3)

### Transformação de dados

In [ ]:
classGender  = LabelBinarizer()
classMaried  = LabelBinarizer()
classResidence = LabelBinarizer()

dataset['gender']       = classGender.fit_transform(dataset['gender'])
dataset['ever_married'] = classMaried.fit_transform(dataset['ever_married'])
dataset['Residence_type'] = classResidence.fit_transform(dataset['Residence_type'])

In [ ]:
dataset = pd.get_dummies(dataset,columns=['smoking_status','work_type'],dtype=int)

In [ ]:
dataset.head(3)

### Limpeza de dados

Removendo o valor other do atributo gender

In [ ]:
dataset.drop( index = dataset.loc[dataset['gender']=='Other'].index , inplace=True)

Removendo atributo  `id` pois ele e chave estrangeira

In [ ]:
dataset.drop(columns='id',inplace=True)

#### Discretização
Tratando os dados nulos , irei transformar atributo numérico bmi em categorico
|             |                |
| ----------- |----------------|
|  < 18.5     | baixo peso     |
| 18.5 - 24.9 | peso ideal     |
| 24.9 - 29.9 | Sobrepeso      |
| 30.0 - 34.9 | obesidade 1    |
| 35.0 - 39.9 | Obesidade (Sévera) |
| 40   >      | Obesidade (Mórbida) |

&nbsp;
> Isso está sendo feito por que os atributos bmi tem outlier (muitos) e para tratar se torna dificil melhor colocar os valores em determinado tipo categoricos de peso corporal

In [ ]:
labels = ['Baixo peso','Peso ideal','Sobrepeso','Obesidade 1','Obesidade 2(sévera)','Obesidade 3(mórbida)']
bins   = [min(data['bmi']),18.5,24.9,29.9,34.9,39.9,max(data['bmi']) ]

In [ ]:
freq = pd.cut(x = dataset['bmi'], bins = bins, labels=labels )

In [ ]:
dataset = pd.concat([freq,dataset.drop(columns='bmi')],axis=1)

In [ ]:
fig = SimpleImputer(strategy='most_frequent')
dataset['bmi'] = fig.fit_transform(dataset)[:,0]

A ideia é imputar dados em atributo `bmi`,contudo atribuir sobrepeso para pessoas com idade <= 18 não faz sentido,pois esses dados em sua maioria tem valores para pessoas que não tiveram Avc. 
Irei também remover  atributo `work_type_children` 

In [ ]:
dataset.loc[( dataset['age']<=18 )]['stroke'].value_counts()

Mostra que dados desse tipo são irrelevantes para classificação

In [ ]:
dataset.loc[dataset['work_type_children']==1]

Crianças que tiveram AVC são muitos poucas 

In [ ]:
index1 = dataset.loc[ ( dataset['age']<=18 )].index
dataset.drop(index=index1,inplace=True)

index2 = dataset.loc[dataset['work_type_children']==1]['stroke'].index
dataset.drop(index=index2,inplace=True)

dataset.drop(columns='work_type_children',inplace=True)

Fazendo LabelEncoder para o atributo `bmi`

In [ ]:
dataset['bmi'] = LabelEncoder().fit_transform(dataset['bmi'])

In [ ]:
dataset[['age','avg_glucose_level','bmi']].corrwith(dataset['stroke'])

In [ ]:
import numpy as np
from scipy.stats import chi2_contingency
def CorrBin(X,Y,namex,namey):
    chi2, p, _, _ = chi2_contingency(pd.crosstab(X,Y))
    if (p>0.1):return f'{namex} <-> {namey}',str(p)
    return f'{namex} <-> {namey}','sair'

In [ ]:
dicti = dict(
    Atributos  =[],
    Correlação =[]
)
for  _ , row in (dataset.nunique().reset_index().iterrows()):
    if row.values[1] == 2:
        a,atr = CorrBin(dataset[row.values[0]],dataset['stroke'],row.values[0],'stroke')
        dicti['Atributos'].append(a)
        dicti['Correlação'].append(atr)

pd.DataFrame(dicti) 

In [ ]:
dataset.drop(columns=['avg_glucose_level'],inplace=True)

Padronização dos dados

In [ ]:
dataset[['age']] = StandardScaler().fit_transform(dataset[['age']])

&nbsp;

&nbsp;

# Classficação

In [ ]:
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV,cross_val_score,StratifiedKFold
from sklearn.neighbors       import KNeighborsClassifier
from sklearn.tree            import DecisionTreeClassifier
from sklearn.linear_model    import LogisticRegression
from sklearn.svm             import SVC
from sklearn.metrics         import *

In [ ]:
from yellowbrick.classifier      import ROCAUC
from yellowbrick.model_selection import LearningCurve

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
X = dataset.drop(columns=['stroke'])
Y = dataset['stroke']

### Balanceamento de dados
Combinando under com o oversampling

In [ ]:
dataset['stroke'].value_counts()

In [ ]:
under = RandomUnderSampler(sampling_strategy={0:320,1:247},random_state=42)
over  = RandomOverSampler(random_state=42)

In [ ]:
Xunder, Yunder = under.fit_resample(X,Y)
Xover , Yover  = over.fit_resample(Xunder,Yunder)

In [ ]:
Yover.value_counts()

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(Xover.values,Yover.values,test_size=0.25,random_state=42) 

## KNN

In [ ]:
params = dict(
    n_neighbors= [5,7,10,12],
    weights   = ['uniform', 'distance'] ,
    algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute'] ,
    leaf_size = [30,50],
    p         = [2,3,5,7],
)

gridKnn = GridSearchCV(
    estimator = KNeighborsClassifier() ,
    param_grid=params,
    scoring='accuracy'
)

In [ ]:
gridKnn.fit(xtrain,ytrain)

In [ ]:
print('Melhor estimador:\n',gridKnn.best_estimator_)

In [ ]:
gridKnn.best_score_

In [ ]:
knn = KNeighborsClassifier(algorithm='ball_tree', n_neighbors=12, p=3)

In [ ]:
knn.fit(xtrain,ytrain)

In [ ]:
pred = knn.predict(xtest)

In [ ]:
plt.figure(figsize=(5,4))
sea.heatmap(confusion_matrix(ytest,pred),annot=True,fmt='')
plt.title(f'accuracy_score {accuracy_score(ytest,pred)}')
plt.show()

In [ ]:
print(f'f1_score :{f1_score(ytest,pred)}')
print(f'recall   :{recall_score(ytest,pred)}')

In [ ]:
print(classification_report(ytest,pred))

In [ ]:
os = ROCAUC(knn)
os.fit(xtrain,ytrain)
os.score(xtest,ytest)
os.show()
plt.show()

In [ ]:
from sklearn.pipeline import Pipeline

## Regressão Logistica

In [ ]:
params = dict(
    penalty=['l1', 'l2', 'elasticnet'],
    solver = ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
    max_iter= [100,350,500,700,1000]
)
gridLogistic = GridSearchCV(
    estimator = LogisticRegression(),
    param_grid= params,
    scoring   = 'accuracy'
) 

In [ ]:
from sklearn.exceptions import FitFailedWarning
warnings.filterwarnings(action='ignore',category=FitFailedWarning)

In [ ]:
gridLogistic.fit(xtrain,ytrain)

In [ ]:
print('Melhor Estimador :\n',gridLogistic.best_estimator_)

In [ ]:
gridLogistic.best_score_

In [ ]:
pred =  LogisticRegression(penalty='l1', solver='liblinear').fit(xtrain,ytrain).predict(xtest)

In [ ]:
plt.figure(figsize=(5,4))
sea.heatmap(confusion_matrix(ytest,pred),annot=True,fmt='')
plt.title(f'accuracy_score {accuracy_score(ytest,pred)}')
plt.show()

In [ ]:
print(f'f1_score :{f1_score(ytest,pred)}')
print(f'recall   :{recall_score(ytest,pred)}')

In [ ]:
print(classification_report(ytest,pred))

## Tree

In [ ]:
params = dict(
    criterion=['gini', 'entropy', 'log_loss'],
    splitter =['best', 'random'] ,
    max_depth= [i for i in range(10,150,20)],
    max_features=['auto', 'sqrt', 'log2'],
)
gridTree = GridSearchCV(
    estimator = DecisionTreeClassifier(),
    param_grid= params,
    scoring   = 'accuracy', 
) 

In [ ]:
gridTree.fit(xtrain,ytrain)

In [ ]:
gridTree.best_score_

### Svm